<a href="https://colab.research.google.com/github/ACM-Research/music-generation/blob/main/Newest_Music_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from music21 import converter, interval, instrument, note, chord, common, stream, midi, tempo
import glob
import concurrent.futures
import pickle 
import numpy as np 
from keras.utils import np_utils 

In [ ]:
# download 
%pwd 
%rm -rf music-generation/ piano_midis/ 
%rm -rf *.tar.gz 
# !git clone https://github.com/ACM-Research/music-generation.git # orig. data 
!wget https://personal.utdallas.edu/~kxs180075/files/pianoonly-xml.tar.gz #anime 
!wget https://personal.utdallas.edu/~kxs180075/files/pianoonly-midi-full.tar.gz 
!wget https://personal.utdallas.edu/~kxs180075/files/jazz-xml.tar.gz #13 jazz 
!wget https://personal.utdallas.edu/~kxs180075/files/bwv806-xml.tar.gz #5 bach 
!wget https://personal.utdallas.edu/~kxs180075/files/simple-xml.tar.gz #5 simple 
!tar -xf pianoonly-xml.tar.gz 
!tar -xf pianoonly-midi-full.tar.gz 
!tar -xf jazz-xml.tar.gz 
!tar -xf bwv806-xml.tar.gz 
!tar -xf simple-xml.tar.gz 
# %mv -v music-generation/piano_midis ./ 

--2021-04-23 04:50:47--  https://personal.utdallas.edu/~kxs180075/files/pianoonly-xml.tar.gz
Resolving personal.utdallas.edu (personal.utdallas.edu)... 129.110.182.249
Connecting to personal.utdallas.edu (personal.utdallas.edu)|129.110.182.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4497283 (4.3M) [application/x-tar]
Saving to: ‘pianoonly-xml.tar.gz’

pianoonly-xml.tar.g 100%[===================>]   4.29M  2.54MB/s    in 1.7s    

2021-04-23 04:50:49 (2.54 MB/s) - ‘pianoonly-xml.tar.gz’ saved [4497283/4497283]

--2021-04-23 04:50:49--  https://personal.utdallas.edu/~kxs180075/files/pianoonly-midi-full.tar.gz
Resolving personal.utdallas.edu (personal.utdallas.edu)... 129.110.182.249
Connecting to personal.utdallas.edu (personal.utdallas.edu)|129.110.182.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4842640 (4.6M) [application/x-tar]
Saving to: ‘pianoonly-midi-full.tar.gz’

pianoonly-midi-full 100%[===================>]   

In [ ]:
def isolate(mfile):
    # print("Reading ", mfile, " = ", end="") 
    orig_midi = converter.parse(mfile) 
    key = orig_midi.analyze("key") 
    if key.tonic == "C": # assume music21 is wrong 
        return None  
    iv = interval.Interval(key.tonic, pitch.Pitch("C")) 
    midi = orig_midi.transpose(iv) 
    # print("transposed to", midi.analyze("key").tonic) 
    midi = midi.flat 
    parts = instrument.partitionByInstrument(midi)
    if parts: 
        print([instr.partName for instr in parts]) 
        for instr in parts: 
            # print(instr.partName, end=", ") 
            if instr.partName and "Piano" in instr.partName: # xml uses variations like "Grand Piano" 
                return [mfile, instr] 

    return None


In [ ]:
# http://cs229.stanford.edu/proj2018/report/18.pdf 
# https://www.dropbox.com/sh/ttzb502hheo9fst/AABIlaxy09RRt2b316jcFwIga/LSTM?dl=0&preview=LSTM.py&subfolder_nav_tracking=1 
# https://mtosmt.org/issues/mto.13.19.3/mto.13.19.3.tymoczko.pdf 
def parse_notes(score):
    score_elems = score.recurse()

    note_list = [] 
    pitches = [] # [[treble, bass], ...] 
    elem_lengths = [] # [[treble, bass], ...] 
    offsets = [] 
    prev_offset = -1 
    for element in score: # score_elems: 
        # print(element, "length", element.duration.quarterLength, "offset", element.offset, "prev", prev_offset) 
        if element.duration.quarterLength <= 0: 
            continue 
        if isinstance(element, note.Note): 
            if element.offset != prev_offset: # this is a note in a brand new time offset 
                note_list.append(str(element.pitch)) 
                pitches.append([element.pitch]) 
                elem_lengths.append([round(float(element.duration.quarterLength), 8)]) 
                offsets.append([float(element.offset)]) 
            else: # this is still in the old time offset 
                if len(pitches[-1]) != 2: # and the old time offset can hold more notes 
                    pitches[-1].append(element.pitch) 
                    elem_lengths[-1].append(round(float(element.duration.quarterLength), 8)) 
                    offsets[-1].append(float(element.offset)) 
            prev_offset = element.offset 
        elif isinstance(element, chord.Chord): 
            if element.offset != prev_offset: 
                temp = '.'.join(str(n.pitch) for n in element)
                note_list.append(temp.split('.')) 
                pitches.append([[k.pitch for k in element]]) # a chord 
                elem_lengths.append([round(float(element.duration.quarterLength), 8)]) 
                offsets.append([float(element.offset)]) 
            else: 
                if len(pitches[-1]) != 2: 
                    pitches[-1].append([k.pitch for k in element]) 
                    elem_lengths[-1].append(round(float(element.duration.quarterLength), 8)) 
                    offsets[-1].append(float(element.offset)) 
            prev_offset = element.offset 
        elif isinstance(element, note.Rest):
            if element.offset != prev_offset: 
                note_list.append(None) 
                pitches.append([None]) 
                elem_lengths.append([round(float(element.duration.quarterLength), 8)]) 
                offsets.append([float(element.offset)]) 
            else: 
                if len(pitches[-1]) != 2: 
                    pitches[-1].append(None) 
                    elem_lengths[-1].append(round(float(element.duration.quarterLength), 8)) 
                    offsets[-1].append(float(element.offset)) 
            prev_offset = element.offset 
        else:
            # print("None of the above", element)
            # prev_offset = element.offset 
            pass 
    # print("parse_notes pitches", pitches) 
    def fixitup(arr, addNone=True): 
        for frame in arr: 
            if len(frame) < 2: 
                if addNone: 
                    frame.append(None) 
                else: 
                    frame.append(frame[0]) # (frame[0]) 
    fixitup(pitches, addNone=True) 
    fixitup(elem_lengths, addNone=False) 
    fixitup(offsets) 
     
    return (note_list, elem_lengths, pitches, offsets) 

In [ ]:
from music21 import pitch 
c = pitch.Pitch(63) 
print(c, pitch.Pitch(63).ps == pitch.Pitch("Eb4").ps == pitch.Pitch('D#4').ps) 
 
def numerical(name): 
    try: 
        return pitch.Pitch(name).ps 
    except Exception: 
        print("Bad name", name)
 

E-4 True


In [ ]:
def pstr(mpitch): 
    #print(mpitch) 
    def octv(k): 
        v = k.octave 
        # v = 4 
        r = pitch.Pitch(k.name+str(v)) 
        return r 
    if not mpitch or (isinstance(mpitch, str) and "r" in mpitch):
        return "r" #frozenset(["r"]) # 
    if isinstance(mpitch, str) and "b" in mpitch: 
        print("b is not allowed!") 
        raise 
        return "r" 
    if isinstance(mpitch, list) or isinstance(mpitch, set): 
        mpitch = [octv(k) for k in mpitch] 
        return "-".join([str(k.ps) for k in mpitch]) # frozenset([str(k.ps) for k in mpitch]) # inverted chords are now considered the same # "-".join # str(k.ps) 
    else: 
        mpitch = octv(mpitch) 
        return str(mpitch.ps) # frozenset([str(mpitch.ps)]) # str(mpitch.ps) 

def map_values(notes, lengths, pitches, offsets, counter): # enum_count, all_names, name_to_num): 
    # lengths, pitches = [k[0] for k in lengths], [k[0] for k in pitches] 
    enum_count, all_names, name_to_num = counter.enum_count, counter.all_names, counter.name_to_num 
    # print(enum_count) 
    # all_names: this song. pstr: converts a pitch into a string. 
    all_names = [pstr(k[0]) for k in pitches] + [pstr(k[1]) for k in pitches] 
    for k in all_names:
      if k not in name_to_num:
          name_to_num[k] = enum_count
          enum_count += 1
    # index = 0 
    vector_list = []
    for index in range(min(len(pitches), len(lengths))): # for item in all_names: # 64-63-23 
        music_vector = []
        length = max(lengths[index][0], lengths[index][1]) # length of this frame 
        # offset = offsets[index]
        # print(item, length) 
        if length <= 8: 
            treble, bass = pstr(pitches[index][0]), pstr(pitches[index][1]) 
            treble_length, bass_length = lengths[index][0], lengths[index][1] 
            music_vector = [name_to_num[treble], treble_length, name_to_num[bass], bass_length] 
            # if "r" not in item: 
            #     music_vector = [name_to_num[item], length] 
            # else: 
            #     music_vector = [0, length] 
            vector_list.append(music_vector)
        else:
            pass # print("Threw away note of length", length) 
        # index += 1 
    # print(vector_list)
    counter.enum_count = enum_count 
    counter.all_names = all_names 
    counter.name_to_num = name_to_num 
    return (vector_list, counter) 
     

In [ ]:
%rm -rvf stream* 

In [ ]:
import sys 
import copy 
import math 
from numpy import argmax 
from keras.utils import normalize, to_categorical 
def qround(value): 
    return math.ceil(value*4)/4 
def sequence_values(counter, normalizers): # mapped_notes, enum_count, name_to_num, 
    mapped_notes, enum_count, name_to_num = counter.mapped_notes, counter.enum_count, counter.name_to_num 
    max_duration = counter.max_duration
    max_offset = counter.max_offset
    seq_size = normalizers.seq_size 
    # mapped_notes = [ song, song, song ] 
    # song = [ [pitch, duration], [pitch, duration], ... ] 
    # print(mapped_notes[0][0:100]) 
    nw_in, nw_out = [], [] 
    all_pkgs = set() 
    # input [100 of [pitch, duration]], output next pitch name
    for vector_list in mapped_notes: 
        # print("origin", vector_list[:10], "...") 
        for idx in range(0, len(vector_list)): 
            music_vector = copy.deepcopy(vector_list[idx]) 
            # print(music_vector, "with enum_count", enum_count, "and max_duration", max_duration)
            # result = (75, 0.6) where 75 => pitch/chord mapping and 0.6 => quarter length 
            result = (music_vector[0], qround(music_vector[1]), 
                      music_vector[2], qround(music_vector[3])) 
            all_pkgs.add(result) 
            # music_vector[2] = music_vector[2]/max_offset
            vector_list[idx] = result 
         
        # print("nmlized", vector_list[:10], "...") 
    pkg_to_int = dict([(pkg, number) for number, pkg in enumerate(list(all_pkgs))]) 
    mean, std = np.mean(list(pkg_to_int.values())), np.std(list(pkg_to_int.values())) 
    print("packaged notes, there are", len(pkg_to_int), "packages. \n\tpkgs=", list(all_pkgs)[0:5], "...") 
    for vector_list in mapped_notes: 
        for k in range(0, len(vector_list) - seq_size - 1): 
            window = vector_list[k:k+seq_size] # [[pitch, duration], ...] 
            expected = vector_list[k+seq_size] # [pitch, duration] 
            # print(window, "\t and expected is", expected, sep="\n") 
            nw_in.append([pkg_to_int[k] for k in window]) 
            nw_out.append(pkg_to_int[expected]) 
    # nw_in, nw_out = None, None 
    # mstream = unmap_values(nw_in[0], counter) 
    # print("Output stream midi file") 
    # mstream.write('midi', fp="stream_"+str(datetime.now())+"_demo.mid") 
    # sys.exit(0) 
    print("Our code is done,", len(nw_in), len(nw_out)) 
    num_pkgs, num_windows, seq_size = len(pkg_to_int), len(nw_in), len(nw_in[0]) 
    normalizers.mpc, normalizers.spc, normalizers.mln, normalizers.sln = 0, 0, 0, 0 
    normalizers.pkg_to_int = pkg_to_int 
    normalizers.mean, normalizers.std = mean, std 
    normalizers.num_pkgs = num_pkgs 
    # print(nw_in[0:100], nw_out[:100], sep="\n") 
    print(num_windows, "windows, each with", seq_size, "frames, (each with", 4, "features that map to 1 number), ", 
        "total=window*frame*1=", num_windows*seq_size*1) 
    nw_in = np.array(nw_in).astype(np.float32) 
    nw_out = np.array(nw_out).astype(np.float32) 
     
    nw_in = np.reshape(nw_in, (num_windows, seq_size, 1)) 
    nw_out = to_categorical(nw_out) # reshapes to (num_pkgs, num_pkgs) 
    # nw_out = np.reshape(nw_out, (num_windows, 4)) 

    print("Input and output shapes:", nw_in.shape, nw_out.shape) 
    return (nw_in, nw_out, counter, normalizers) 
     

In [ ]:
!pip install keras-self-attention 
from keras.models import Sequential 
from keras.layers import Activation, Dense, Bidirectional, LSTM, Dropout, Flatten, TimeDistributed, Reshape 
from keras.callbacks import ModelCheckpoint 
from keras.optimizers import SGD 
from keras.optimizers import Adam 
from keras_self_attention import SeqSelfAttention
# https://stackoverflow.com/a/37213763 
def basic_network(counter, normalizers): # * 
    model = Sequential() 
    # model.add(Bidirectional(LSTM(256, input_shape=(seq_size, 4), return_sequences=True))) # 256 
    model.add(LSTM(512, input_shape=(normalizers.seq_size, 1), return_sequences=True)) 
    # model.add(TimeDistributed(Dense(4))) 
    model.add(SeqSelfAttention(attention_activation='linear')) # linear 
    model.add(LSTM(512, input_shape=(normalizers.seq_size, 1), return_sequences=False)) 
    model.add(Flatten()) 
    model.add(Dense(normalizers.num_pkgs)) 
    model.add(Activation('softmax')) # * linear, softmax, sigmoid ? 
    opt = Adam(learning_rate=1e-4) # SGD(lr=1e-3) # Adam(learning_rate=0.005) # opt = SGD(lr=0.005) # 1e-3) # * 
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']) # loss mae, mse, categorical_crossentropy, opt rmsprop 
    # print("Building model......")
    # model = Sequential() # initialize a sequential model 
    # model.add(LSTM(512,return_sequences=False, input_shape=(100, 3))) # 2nd layer of LSTM 
    # model.add(Dropout(0.75)) # 2nd layer of Dropout
    # model.add(Dense(3)) # a dense layer of 3 tensors 
    # model.add(Activation('linear'))
    # model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model 
def load_model(filename, counter, normalizers): 
    model = basic_network(counter, normalizers) 
    model.load_weights(filename) 
    return model 
def train(model, nw_in, nw_out, num_epochs=100): 
    fp = "weights.best.hdf5" 
    checkpoint = ModelCheckpoint(fp, monitor='loss', verbose=1, save_best_only=True) 
    model.fit(nw_in, nw_out, epochs=num_epochs, batch_size=256, callbacks=[checkpoint]) 




  Created wheel for keras-self-attention: filename=keras_self_attention-0.49.0-cp37-none-any.whl size=19468 sha256=f9305b951e1ac797f11f8ede8cdd1dd50c5044c48a6c841c288d2b68e23af859
  Stored in directory: /root/.cache/pip/wheels/6f/9d/c5/26693a5092d9313daeae94db04818fc0a2b7a48ea381989f34
Successfully built keras-self-attention


In [ ]:
def unmap_values(complete, counter, normalizers):
    # input: list of music vectors, counter object
    # output: music21 stream object
    # music_stream = stream.Stream()
    print("spc: ", normalizers.spc)
    print("mpc: ", normalizers.mpc)
    print("sln: ", normalizers.sln)
    print("mln: ", normalizers.mln)
    top, bottom = stream.Part(), stream.Part() 
    int_to_pkg = {num:name for name, num in normalizers.pkg_to_int.items()} 
    print("int_to_pkg len ", len(int_to_pkg), ": ", int_to_pkg) 
    num_to_name = {num:name for name, num in counter.name_to_num.items()}
    off, count = 0, 0 
    top.insert(0, instrument.Piano()) 
    bottom.insert(0, instrument.Piano()) 
    # print(num_to_name)
    def add_to_stream(category, is_treble, i, j, off=0): 
        music_vector = int_to_pkg[category] # actually a tuple 
        pitch, length = music_vector[i], music_vector[j] 
        # pitch, length = music_vector[i], music_vector[j]

        # # doing this ruins normalisation
        # pitch = abs(pitch)
        # length = abs(length) 

        # pitch = pitch*normalizers.spc + normalizers.mpc
        # length = float(length*normalizers.sln + normalizers.mln)
        
        pitch = round(pitch)
        if length < 0 or pitch < 0:
            print("ERR ", is_treble, ")", [length, pitch])
            return
        # offset = music_vector[2] * counter.max_offset
        # print(music_vector) 
        if length > 8: 
            length = 0.5 
        if pitch not in num_to_name.keys(): 
            print("ERR", is_treble, ")", category, "=>", music_vector, "=>", [pitch, length], "=>", "?") 
            return 
        elem_name = num_to_name[pitch] # num_to_name[pitch], "-".join(list(num_to_name[pitch])) 
        if elem_name == 'b': 
            return # refuse to add anything to stream 
        if elem_name == 'r':
            elem = note.Rest() 
        elif '-' not in elem_name:
            elem = note.Note(int(float(elem_name)))
        else:
            pitch_arr = elem_name.split('-')
            pitch_arr = [int(float(x)) for x in pitch_arr]
            elem = chord.Chord(pitch_arr)
            
        elem.storedInstrument = instrument.Piano()
        elem.duration.quarterLength = length
        # elem.offset = off 
        if is_treble: 
            top.insert(off, elem) # top.append(elem) # top.insert(off, elem) 
        else: 
            bottom.insert(off, elem) # bottom.append(elem) # bottom.insert(off, elem) 
        print(is_treble, ")", category, "=>", music_vector, "=>", [pitch, length], "=>", 
              elem, ", length:", elem.duration.quarterLength, ", off:", off, ", offset:", elem.offset) 
        return elem 

    for category in complete: 
        # while len(music_vector) < 4: 
        #     music_vector.append(0.5) 
        tre = add_to_stream(category, True, 0, 1, off) 
        bae = add_to_stream(category, False, 2, 3, off) 
        off += 0.5 
        #off += max(0.5, 
                   #max(tre.duration.quarterLength, bae.duration.quarterLength)) 
        # music_stream.append(elem) 
        count += 1 

    music_stream = stream.Stream([top, bottom]) 
    return music_stream

from datetime import datetime 
def stream_file(stream, title=""): # todo use array of streams 
    %mkdir -p stream_out 
    %cd stream_out 
    stream.write('midi', fp="stream_"+str(datetime.now())+"_demo"+title+".mid") 
    %cd .. 
     

In [ ]:
import copy 
# import tensorflow as tf
def use_model(model, nw_in, counter): 
    # nw_in is [window, window, ...] 
    start = int(len(nw_in)/4) # 0 # int(len(nw_in)/2) # 0 
    pattern_in = list(nw_in[start]) # python list of ndarrays dtype=float32. do not convert ndarray to python list. len of pattern_in is sequence size. 
    seq_size = len(pattern_in) 
    complete = list(nw_in[start]) # .tolist() 
    print("starting sequence window", complete[0:10], "...") # pattern_in) 
    for idx in range(200): 
        print("generating note", idx, "...") 
        # print("\tPY INPUT", pattern_in[0:3], "...") 
        ptn = np.asarray(pattern_in) 
        prediction_input = np.reshape(ptn, (1, seq_size, -1)) # np.reshape(ptn, (1, len(ptn), 2)) # np.array(pattern_in)
        # print("\tINPUT", prediction_input, sep="\n") 
        # print(type(prediction_input), type(prediction_input[0]), type(prediction_input[0][0]), type(prediction_input[0][0][0]))
        
        # print("\tINPUT SHAPE", prediction_input.shape) 
        prediction = model.predict(prediction_input, verbose=0) # confidences where each index is a category 
        num_cat = len(prediction[0]) 
        # print("\tPREDICTIONS", prediction) 
        predicted_next = np.argmax(prediction) 
        if idx % 4 == 0: 
            predicted_next = np.random.choice(num_cat, 1, p=prediction[0])[0] # np.argmax(prediction) # highest conf. index # .tolist() 
        # print("\tOUTPUT", predicted_next) 
        complete.append(np.asarray(predicted_next, dtype='float32').reshape(1)) # numpy array reshape into list. so 419 => nparray(419.0) => nparray([419.0]) 
        # print("\tSO FAR", complete[:2], "...", complete[-2:]) 
        # pattern_in += predicted_next # np.concatenate(pattern_in, predicted_next) 
        pattern_in = complete[idx:idx+seq_size] #len(complete) - 1] # pattern_in[1:len(pattern_in)] 
    return [int(k) for k in complete] # int() will attempt to coerce numpy predictions. python list of ints. 

In [ ]:
def begin_train(nw_in, nw_out, counter, normalizers): 
    model = basic_network(counter, normalizers) 
    model.summary() 
    train(model, nw_in, nw_out, num_epochs=500) 

In [ ]:
import os.path 
import math 
import sys 
from datetime import datetime 
# read data -> 
# map input (note:num) -> 
# sequence input (num to 0..1) then (window to next) -> 
# package network input it into 1 pickle -> 
# train -> 
# save best into hdf5 -> 
# use hdf5 to generate new notes -> 
# unmap (0..1 to num) then (num:note) -> 
# add to stream -> 
# output midi file 
count = 0 
cpus = 16 
debug, stop_count = False, 20 
nw_pickle, search = "nw.pkl", './pianoonly-xml/' # "./pianoonly-xml/" # "./simple-xml/" # "./chorales/xml/" # "./pianoonly-xml/"
normalize_pickle = "normalize.pkl"
!rm -v music21parse.mid unmap_demo.mid 

class Counters:
    def __init__(self):
        self.enum_count = 0 
        self.unique_frames = 0 
        self.all_names = [] 
        self.name_to_num = {} 
        self.mapped_notes = [] 
        self.max_duration = 0
        self.max_offset = 0

class Normalization:
    def __init__(self): 
        self.pkg_to_int = dict() 
        self.num_pkgs = 0 
        self.mean = 0 
        self.std = 0 
        self.seq_size = 64 
        self.mpc = 0 
        self.spc = 0 
        self.mln = 0 
        self.sln = 0 
     
c = Counters()
n = Normalization()
if os.path.isfile(nw_pickle): 
    print("this is just to let you know that the network pickle file is being used, press any key to confirm") 
    consume = input() 
    print("reading from network pickle...") 
    nwfile = open(nw_pickle, "rb") 
    print("(1/2) loading...") 
    nw = pickle.load(nwfile) 
    print("(2/2) parsing...") 
    nw_in, nw_out, c.enum_count, c.max_duration, c.name_to_num = nw[0], nw[1], nw[2], nw[3], nw[4] 
    nwfile.close() 
    print("nw_out:", list(nw_out[:3]), "...") 
    print("nw_out has", len(nw_out), "notes, about", math.ceil(len(nw_out)/450), "songs") 
    print("enum_count", nw[2], "max_duration", nw[3]) 
    print("name_to_num has keys", list(nw[4].keys())[:5], "...") 
    print("done, starting to train...")

    print("reading from normalizers pickle...")
    nrmlfile = open(normalize_pickle, "rb")
    print("(1/2) loading...") 
    n = pickle.load(nrmlfile)
    print("(2/2) parsing...") 
    print("there are", len(n.pkg_to_int), "categories") 
    # begin_train(nw_in, nw_out) 
    # exit(0)  
    if os.path.isfile("weights.best.hdf5"): 
        print("generating notes...")
        ###########
        # use the model for prediction 
        # 
        model = load_model("weights.best.hdf5", c, n) 
        result = use_model(model, nw_in, c) 
        print("RESULT", result) 
        result_stream = unmap_values(result, c, n) 
        print("Output result midi file")
        result_stream.write('midi', fp="generated_"+str(datetime.now())+"_demo.mid")
        raise 
    else: 
        begin_train(nw_in, nw_out, c, n) 
        raise 
 
# otherwise, do map and sequencing ############################################ 
print("no pickle") 
all_files = sorted(glob.glob(search+"*.mid") + glob.glob(search+"*.xml")) # piano_midis 
stop_count = len(all_files) if stop_count < 0 or stop_count > len(all_files) else stop_count 
all_files = all_files[0:stop_count] 
all_pitches = []
 
print("-----", "(", "debug" if debug else "", len(all_files), "files", ")", "-----", end="\n\n") 
 
# one worker per core
# with concurrent.futures.ThreadPoolExecutor(max_workers=cpus) as executor:
    # for ret in executor.map(isolate, all_files): 
for mfile in all_files: 
    ret = isolate(mfile) 
    count += 1 
    if ret: 
        print("(", count, "/", stop_count, ") ", ret[0], sep="") 
        piano_part = ret[1] 
        if count == 1: 
            piano_part.write('midi', fp="music21parse.mid") 
        notes, lengths, pitches, offsets = parse_notes(piano_part) 
        # fixme shouldn't add pitches like this because they'll overlap with the window regardless of the song..., and it's super bad 
        # try: 
        all_pitches.extend(pitches) 
        # print(len(notes), len(lengths), len(pitches))
        vector_list, new_c = map_values(notes, 
                                  lengths, 
                                  pitches,
                                  offsets,
                                  c) 
        c = new_c 
        c.mapped_notes.append(vector_list) 

max_duration = 0
for vector_list in c.mapped_notes:
    for music_vector in vector_list: 
        # print(music_vector, max_duration) 
        if music_vector[1] > max_duration: 
            # print(music_vector) 
            max_duration = music_vector[1]
c.max_duration = max_duration

# max_offset = 0
# for vector_list in c.mapped_notes:
#     for music_vector in vector_list: 
#         # print(music_vector, max_duration) 
#         if music_vector[2] > max_offset: 
#             # print(music_vector) 
#             max_offset = music_vector[2]
# c.max_offset = max_offset
if debug: 
    print(c.mapped_notes) 
    print("sequence_values on", c.mapped_notes, c.enum_count, c.name_to_num, max_duration, sep="\n") 

# Testing the unmap notes function
# Problem is with notes being overlaid. {offset} rather than length? or offset and length? not sure
# test_output = c.mapped_notes[0]
# print(test_output)
# for music_vector in test_output:
#     music_vector[0] /= c.enum_count
#     music_vector[1] /= c.max_duration
#     # music_vector[2] /= c.max_offset 

# music_stream = unmap_values(test_output, c)

# # print(music_stream.show('text'))

# print("Outputting test midi file...")
# music_stream.write('midi', fp="unmap_demo.mid")


 
nw_in, nw_out, new_c, new_n = sequence_values(c, n) 
c = new_c 
n = new_n
#print(nw_in[40:50]) 
# vocab = set([pstr(k) for k in all_pitches]) 
# print("done, saw", len(all_pitches), "pitches,", len([k for k in vocab if "-" in k]), "unique chords and", 
#        len([k for k in vocab if "-" not in k]), "unique pitches (excluding chords)") 
exit(1) 
############################################################################### 
# save into pickle 
# !rm -rvf 
print("saving into sequences pickle...") 
nwfile = open(nw_pickle, 'wb') 
print("(1/2) network...") 
nw = [nw_in, nw_out, c.enum_count, c.max_duration, c.name_to_num] 
print("(2/2) dumping...") 
pickle.dump(nw, nwfile) 
nwfile.close() 

print("saving into normalizers pickle...")
nrmlfile = open(normalize_pickle, 'wb')
pickle.dump(n, nrmlfile)
print("done, starting to train...") 
begin_train(nw_in, nw_out, c, n) 


rm: cannot remove 'music21parse.mid': No such file or directory
rm: cannot remove 'unmap_demo.mid': No such file or directory
no pickle
----- (  20 files ) -----

['Grand Piano']
(1/20) ./pianoonly-xml/ACruelAngelsThesis.mid.xml
['Grand Piano, CFX Concert Grand 1']
(2/20) ./pianoonly-xml/ADAMAS(Theishter).mid.xml
['Grand Piano, Piano']
(3/20) ./pianoonly-xml/ADAMASOG.mid.xml
['Grand Piano, SmartMusic SoftSynth']
(4/20) ./pianoonly-xml/AOHARUOP.mid.xml
['Bright Piano, Solo']
(5/20) ./pianoonly-xml/AOTOP2.mid.xml
['Grand Piano, Piano']
(6/20) ./pianoonly-xml/APageofMySto…PrincipalED).mid.xml
['Grand Piano, Piano']
(7/20) ./pianoonly-xml/AQUA.mid.xml
['Grand Piano']
(8/20) ./pianoonly-xml/ARIA_The_ORIGINA…achi-_Spirale.mid.xml
['Grand Piano, Piano']
(9/20) ./pianoonly-xml/ATenderFeeling.mid.xml
['Grand Piano, New Instrument']
(10/20) ./pianoonly-xml/A_Cruel_Angels_Thesis.mid.xml
['Grand Piano']
(11/20) ./pianoonly-xml/A_Tender_Feeling…sbeths_Theme].mid.xml
['Grand Piano']
(12/20) ./pianoo

In [ ]:
# !rm -rvf generated* stream* 